In [24]:
##################################################################################
## Code to create the input digital models required for SOLWEIG: DEM, DSM, CDSM ##
##################################################################################

## Import the required libraries
import pdal
import numpy as np
import json
import rasterio
from rasterio.transform import from_origin
from rasterio import Affine
from rasterio.enums import Resampling
import os
from osgeo import gdal
import shutil
import glob
import math
from pyproj import Transformer # for translating coordinates from nad(83) 2011 to lat lon

## Define all the functions to be used

#### Defining functions for merging and interpolating to fill gaps in DEM

def merge_rasters(input_files, output_file, nodata_value=-9999, data_type=gdal.GDT_Float32):
    """
    Merges multiple raster files into a single raster using GDAL.

    Parameters:
        input_files (list): List of input raster file paths to merge.
        output_file (str): Path to the output merged raster file.
        nodata_value (float): NoData value for input and output rasters.
        data_type (GDAL DataType): Data type for the output raster (e.g., gdal.GDT_Float32).
        
    Returns:
        None
    """
    
    # Set the options for gdal.Warp
    warp_options = gdal.WarpOptions(
        format='GTiff',
        srcNodata=nodata_value,
        dstNodata=nodata_value,
        xRes=None,  # Keeps the original resolution
        yRes=None,
        creationOptions=['COMPRESS=LZW'],  # Compression option (optional)
        outputType=data_type
    )
    
    # Execute the merge operation
    gdal.Warp(output_file, input_files, options=warp_options)
    print(f"Raster files merged and saved to {output_file}")


### Filling in the gaps in the DSM
## using inverse weighting (idw)

def fill_nodata(input_dsm_path, output_dsm_path, max_distance=100, smoothing_iterations=0):
    """
    Fills NoData values in a DSM using GDAL's FillNodata function.

    Parameters:
        input_dsm_path (str): Path to the input DSM file.
        output_dsm_path (str): Path to save the output filled DSM file.
        max_distance (int): Maximum search distance (in pixels) to find values for interpolation.
        smoothing_iterations (int): Number of smoothing iterations to perform on the filled area.

    Returns:
        None
    """
    # Open the input DSM
    dsm_ds = gdal.Open(input_dsm_path, gdal.GA_ReadOnly)
    if dsm_ds is None:
        print("Failed to open the input DSM.")
        return

    # Get raster band (assumes DSM has one band)
    dsm_band = dsm_ds.GetRasterBand(1)

    # Create output DSM with the same dimensions and georeferencing
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(
        output_dsm_path,
        dsm_ds.RasterXSize,
        dsm_ds.RasterYSize,
        1,
        gdal.GDT_Float32
    )

    # Copy georeference and projection from the input DSM
    output_ds.SetGeoTransform(dsm_ds.GetGeoTransform())
    output_ds.SetProjection(dsm_ds.GetProjection())

    # Read data and copy to the output band
    output_band = output_ds.GetRasterBand(1)
    dsm_data = dsm_band.ReadAsArray()

    # Set NoData value (assumes NoData value is already set in the input band)
    nodata_value = dsm_band.GetNoDataValue()
    output_band.SetNoDataValue(nodata_value)
    output_band.WriteArray(dsm_data)

    # Fill NoData values using GDAL's FillNodata function
    gdal.FillNodata(
        targetBand=output_band,
        maskBand=None,
        maxSearchDist=max_distance,
        smoothingIterations=smoothing_iterations
    )

    # Flush changes and close datasets
    output_band.FlushCache()
    output_ds = None
    dsm_ds = None
    print(f"Filled DSM saved to {output_dsm_path}")

## check if all of the gaps in the data have been filled

def check_nodata(dsm_path, nodata_value=-9999):
    """
    Checks if the DSM has any NoData values.

    Parameters:
        dsm_path (str): Path to the DSM file.
        nodata_value (float): NoData value to check for.
    
    Returns:
        bool: True if there are NoData values, False otherwise.
    """
    # Open the DEM
    dsm_ds = gdal.Open(dsm_path, gdal.GA_ReadOnly)
    if dsm_ds is None:
        print("Failed to open the DEM.")
        return False

    # Get the raster band and read the data as an array
    dsm_band = dsm_ds.GetRasterBand(1)
    dsm_data = dsm_band.ReadAsArray()

    # Check if any NoData values are present
    has_nodata = np.any(dsm_data == nodata_value)
    dsm_ds = None  # Close the dataset
    return has_nodata

# keep filling the gaps until there are none left

def fill_gaps_until_complete(input_dsm, output_dsm, max_distance=100, smoothing_iterations=1):
    """
    Repeatedly fills NoData values in a DSM until no gaps remain.

    Parameters:
        input_dsm (str): Path to the input DSM file.
        output_dsm (str): Path to the output DSM file.
        max_distance (int): Maximum search distance for filling gaps.
        smoothing_iterations (int): Smoothing iterations during fill process.
    """
    
    iteration = 1
    fill_nodata(input_dsm, output_dsm, max_distance, smoothing_iterations)
    paths = [output_dsm]
    new_input_dsm_path = output_dsm
    if not check_nodata(new_input_dsm_path, nodata_value=-9999):
        new_output_dsm_path = new_input_dsm_path
    # Continue filling gaps until no NoData values remain
    while check_nodata(new_input_dsm_path, nodata_value=-9999):
        print(f"Iteration {iteration}: Filling NoData values...")
        new_input_dsm_path = paths[iteration - 1]
        new_output_dsm_path = os.path.join(classifications_folder, 'DEM_python_iteration{}.tif'.format(iteration))
        paths.append(new_output_dsm_path)
        fill_nodata(new_input_dsm_path, new_output_dsm_path, max_distance, smoothing_iterations)
        iteration += 1
    
    print("No more NoData values remain. Filling process completed.")
    return new_output_dsm_path

## defining function for converting the dms from feet to meters 

def convert_dsm_feet_to_meters(input_dsm_path, output_dsm_path):
    """
    Converts a DSM raster from feet to meters.

    Parameters:
        input_dsm_path (str): Path to the input DSM file in feet.
        output_dsm_path (str): Path to the output DSM file in meters.

    Returns:
        None
    """
    # Open the input DSM
    dsm_ds = gdal.Open(input_dsm_path, gdal.GA_ReadOnly)
    if dsm_ds is None:
        print("Failed to open the input DSM.")
        return

    # Get the raster band and NoData value
    dsm_band = dsm_ds.GetRasterBand(1)
    nodata_value = dsm_band.GetNoDataValue()
    dsm_data = dsm_band.ReadAsArray()

    # Conversion factor from feet to meters
    feet_to_meters = 0.3048

    # Convert DSM values from feet to meters
    dsm_data_meters = np.where(dsm_data != nodata_value, dsm_data * feet_to_meters, nodata_value)

    # Create the output DSM with the same dimensions and georeferencing
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(
        output_dsm_path,
        dsm_ds.RasterXSize,
        dsm_ds.RasterYSize,
        1,
        gdal.GDT_Float32
    )

    # Copy geotransform and projection from the input DSM
    output_ds.SetGeoTransform(dsm_ds.GetGeoTransform())
    output_ds.SetProjection(dsm_ds.GetProjection())

    # Write the converted data to the output file
    output_band = output_ds.GetRasterBand(1)
    output_band.SetNoDataValue(nodata_value)
    output_band.WriteArray(dsm_data_meters)

    # Flush changes and close datasets
    output_band.FlushCache()
    output_ds = None
    dsm_ds = None
    print(f"DSM successfully converted to meters and saved to {output_dsm_path}")





In [20]:
# get the laz files in the central area of Durham
file_path = "D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles"  
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(file_path):
    if file_name.endswith(".laz"):
        file_name_list.append(file_name)


In [ ]:
Starting processing for Job1021395_35078_98_88.laz

In [37]:
file_name_list = file_name_list[1:]

In [28]:
file_name_list

['Job1021395_35078_98_88.laz',
 'Job1021395_35078_98_91.laz',
 'Job1021395_35078_98_94.laz',
 'Job1021395_36078_01_88.laz',
 'Job1021395_36078_01_91.laz',
 'Job1021395_36078_01_94.laz']

In [4]:
# need to figure out whats happened with below file
#'Job1021395_35078_98_88.laz'

In [38]:

# for the chosen file, extract required info to build the dms
# Classify either ground, roads, trees or buildings
# Ground = 2, Roads = 13, Trees = 5, Buildings = 6

classifications_folder = 'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/'

classify_list = ['ground','roads','trees','buildings']

for laz_filename in file_name_list:
    print('Starting processing for', laz_filename)
    for classify in classify_list:
        if classify == 'ground':
            class_num = 2
        if classify == 'roads':
            class_num = 13
        if classify == 'trees':
            class_num = 5
        if classify == 'buildings':
            class_num = 6

        print('Classification:',classify)
        # Input and output files
        input_las = file_path+'\\'+laz_filename
        output_dem = file_path+'\\Classifications\\'+classify+'_'+laz_filename[:-3]+'tif'

        fileinfo = laz_filename[:-4]

        # PDAL Pipeline JSON for processing
        pipeline = {
                "pipeline": [
                {
                    "type": "readers.las",
                    "filename": input_las
                },
                {
                    "type": "filters.range",
                    "limits": "Classification[{}:{}]".format(class_num,class_num)  # remember to set the classification type above!
                },
                {
                    "type": "filters.ferry",
                    "dimensions": "Z=HeightAboveGround"  # Optional: Rename Z to HeightAboveGround
                },
                {
                    "type": "writers.gdal",
                    "filename": output_dem,
                    "resolution": 1.0,  # Define resolution 1-meter grid
                    "output_type": "idw",  # value for the pixel idw is shepards inverse distance weighting for points in radius, mean, min, max as expected
                    "data_type": "float32",
                    "nodata": -9999
                }
            ]
        }

        # Convert the pipeline to a JSON string
        pipeline_json = json.dumps(pipeline)

        # Execute the pipeline
        p = pdal.Pipeline(pipeline_json)

        # Execute the pipeline
        try:
            p.execute()
            print("DM generation successful.")
        except RuntimeError as e:
            print("PDAL pipeline execution failed:", e)

# Load the DEM using Rasterio to check metadata (optional)
#with rasterio.open(output_dem) as dem:
#    print(f"DEM Metadata:\n {dem.meta}")
#    print("DEM preview (5x5):")
#    data = dem.read(1)  # Read the first band
#    print(data[:5, :5])

    ##################################################################
    ##################### CREATING THE DEM ###########################
    ##################################################################

    ### merge the ground and road layers
    ground_filename = 'ground_{}.tif'.format(fileinfo)
    road_filename = 'roads_{}.tif'.format(fileinfo)
    ground_filepath = os.path.join(classifications_folder, ground_filename)
    road_filepath = os.path.join(classifications_folder, road_filename)

    input_files = [ground_filepath, road_filepath]

    output_file = os.path.join(classifications_folder, 'merged_output.tif')

    merge_rasters(input_files, output_file, nodata_value=-9999, data_type=gdal.GDT_Float32)

    ## fill in the missing data in the merged ground+road layer
    input_dsm_path = output_file
    output_dsm_path =  os.path.join(classifications_folder, 'DEM_python.tif')

    #fill_nodata(input_dsm_path, output_dsm_path, max_distance=100, smoothing_iterations=1)
    #check_nodata(output_dsm_path, nodata_value=-9999)
    final_dem_path = fill_gaps_until_complete(input_dsm_path, output_dsm_path, max_distance=100, smoothing_iterations=1)

    # copy the filled DEM into folder for DEMs
    #final_dem_path = os.path.join(classifications_folder, 'DEM_python_iteration{}.tif'.format(iteration))
    folder_for_final_dems = os.path.join(classifications_folder,'DEMs')
    DEM_filename = 'DEM_{}.tif'.format(fileinfo)
    DEM_filepath = os.path.join(folder_for_final_dems, DEM_filename)
    shutil.copy(final_dem_path, DEM_filepath)

    ##################################################################
    ##################### CREATING THE DSM ###########################
    ##################################################################

    ### merge the DEM and buildings
    buildings_filename = 'buildings_{}.tif'.format(fileinfo)
    buildings_filepath_unsmoothed = os.path.join(classifications_folder, buildings_filename)
    buildings_filepath = os.path.join(classifications_folder, 'buildings', buildings_filename)
    
    fill_nodata(buildings_filepath_unsmoothed, buildings_filepath, max_distance=5, smoothing_iterations=1) ## get rid of the gaps in the buildings

    input_files = [DEM_filepath, buildings_filepath]

    folder_for_final_dsms = os.path.join(classifications_folder,'DSMs')
    DSM_filename = 'DSM_{}.tif'.format(fileinfo)

    DSM_filepath = os.path.join(folder_for_final_dsms, DSM_filename)

    merge_rasters(input_files, DSM_filepath, nodata_value=-9999, data_type=gdal.GDT_Float32)

    ##################################################################
    ##################### CREATING THE CDSM ###########################
    ##################################################################

    ### merge the DEM and tree canopy
    trees_filename = 'trees_{}.tif'.format(fileinfo)
    trees_filepath = os.path.join(classifications_folder, trees_filename)


    ### merge the DEM and trees
    input_files = [ DEM_filepath, trees_filepath]

    folder_for_trees_DEM = os.path.join(classifications_folder, 'trees_plus_DEM')
    trees_DEM_filename = 'trees_DEM_{}.tif'.format(fileinfo)
    trees_DEM_filepath = os.path.join(folder_for_trees_DEM, trees_DEM_filename)

    merge_rasters(input_files, trees_DEM_filepath, nodata_value=-9999, data_type=gdal.GDT_Float32)

    # subtract the DEM from the trees+DEM raster
    # add in condition that values over 150 ft are filtered out, as these are outliers 
    # path to save output to
    CDSM_filename = 'CDSM_{}.tif'.format(fileinfo)
    folder_for_final_cdsms = os.path.join(classifications_folder,'CDSMs')
    CDSM_filepath = os.path.join(folder_for_final_cdsms, CDSM_filename)

    # Define the maximum threshold value for the tree heights (NOTE: this is in feet not meters!!)
    threshold_value = 150

    # Open the raster files
    with rasterio.open(trees_DEM_filepath) as src1, rasterio.open(DEM_filepath) as src2:
        # Check if both rasters have the same shape
        if src1.width != src2.width or src1.height != src2.height:
            raise ValueError("Rasters must have the same dimensions.")
    
        # Read the first band from each raster
        tree_DEM_raster = src1.read(1)
        DEM_raster = src2.read(1)

        # Perform raster subtraction (handling NoData values)
        nodata = src1.nodata   # get the values for no data, should be -9999
        result = np.where(
            (tree_DEM_raster == nodata) | (DEM_raster == nodata), nodata, tree_DEM_raster - DEM_raster
        )
    
        # Apply threshold filtering: Set values greater than 150 to NoData
        result = np.where(result > threshold_value, nodata, result)
    
        # Define the metadata for the output raster
        out_meta = src1.meta.copy()
        out_meta.update({
            "dtype": "float32",
            "nodata": nodata
        })

        # Write the result to a new raster file
        with rasterio.open(CDSM_filepath, "w", **out_meta) as dst:
            dst.write(result, 1)

    print(f"Raster subtraction completed. Output saved as {CDSM_filepath}.")

Starting processing for Job1021395_35078_98_91.laz
Classification: ground
DM generation successful.
Classification: roads
DM generation successful.
Classification: trees
DM generation successful.
Classification: buildings
DM generation successful.
Raster files merged and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/merged_output.tif
Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEM_python.tif
Iteration 1: Filling NoData values...
Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEM_python_iteration1.tif
Iteration 2: Filling NoData values...
Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEM_python_iteration2.tif
No more NoData values remain. Filling process completed.
Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/buildings\buildings_Job1021395_35078_98_91.tif
Raster fi

In [30]:
trees_DEM_filepath

'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/trees_plus_DEM\\trees_DEM_Job1021395_35078_98_88.tif'

In [31]:
DEM_filepath

'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEMs\\DEM_Job1021395_35078_98_88.tif'

In [34]:
with rasterio.open(trees_DEM_filepath) as src1, rasterio.open(DEM_filepath) as src2:
# Check if both rasters have the same shape
    print(src1.width , src2.width, src1.height ,src2.height)

8889 8888 10932 10932


In [7]:
classifications_folder =  'D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications'
## fill in the missing data 
input_dsm_path = 'D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\DEM_Job1021395_36078_01_94.tif'
output_dsm_path =  os.path.join(classifications_folder, 'DEM_python.tif')

check_nodata(input_dsm_path, nodata_value=-9999)
iteration = 1
new_output_dsm_path = os.path.join(classifications_folder, 'DEM_python_iteration{}.tif'.format(iteration))
fill_nodata(input_dsm_path, new_output_dsm_path, max_distance = 100, smoothing_iterations = 2)


<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_14692\3147205706.py:1: SyntaxWarning: invalid escape sequence '\C'
  classifications_folder =  'D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_14692\3147205706.py:3: SyntaxWarning: invalid escape sequence '\C'
  input_dsm_path = 'D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\DEM_Job1021395_36078_01_94.tif'


Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEM_python_iteration1.tif


In [20]:
## merge the DEM and CDSM to get the TDSM files 
CDSM_folder = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs'
DEM_folder = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs'
TDSM_folder = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs'

file_name_list = []
for file_name in os.listdir(CDSM_folder):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)


for CDSM_filename in file_name_list:
# subtract the DEM from the trees+DEM raster
    # add in condition that values over 150 ft are filtered out, as these are outliers 
    # path to save output to
    CDSM_filepath = os.path.join(CDSM_folder, CDSM_filename)
    DEM_filename = 'DEM'+CDSM_filename[4:]
    DEM_filepath = os.path.join(DEM_folder, DEM_filename)
    TDSM_filename = 'TDSM'+CDSM_filename[4:]
    TDSM_filepath = os.path.join(TDSM_folder, TDSM_filename)
    # Open the raster files
    with rasterio.open(CDSM_filepath) as src1, rasterio.open(DEM_filepath) as src2:
        # Check if both rasters have the same shape
        if src1.width != src2.width or src1.height != src2.height:
            raise ValueError("Rasters must have the same dimensions.")
    
        # Read the first band from each raster
        CDSM_raster = src1.read(1)
        DEM_raster = src2.read(1)

        # Perform raster subtraction (handling NoData values)
        nodata = src1.nodata   # get the values for no data, should be -9999
        result = np.where(
            (CDSM_raster == nodata) , DEM_raster, CDSM_raster + DEM_raster
        )
    
        # Define the metadata for the output raster
        out_meta = src1.meta.copy()
        out_meta.update({
            "dtype": "float32",
            "nodata": nodata
        })

        # Write the result to a new raster file
        with rasterio.open(TDSM_filepath, "w", **out_meta) as dst:
            dst.write(result, 1)

    print(f"Raster addition completed. Output saved as {TDSM_filepath}.")

<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:3: SyntaxWarning: invalid escape sequence '\l'
<>:4: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:3: SyntaxWarning: invalid escape sequence '\l'
<>:4: SyntaxWarning: invalid escape sequence '\l'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_11496\706420290.py:2: SyntaxWarning: invalid escape sequence '\l'
  CDSM_folder = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_11496\706420290.py:3: SyntaxWarning: invalid escape sequence '\l'
  DEM_folder = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_11496\706420290.py:4: SyntaxWarning: invalid escape sequence '\l'
  TDSM_folder = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs'


Raster addition completed. Output saved as D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\TDSM_Job1021395_35078_95_88.tif.
Raster addition completed. Output saved as D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\TDSM_Job1021395_35078_95_91.tif.
Raster addition completed. Output saved as D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\TDSM_Job1021395_35078_95_94.tif.
Raster addition completed. Output saved as D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\TDSM_Job1021395_35078_98_88.tif.
Raster addition completed. Output saved as D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\TDSM_Job1021395_35078_98_91.tif.
Raster addition completed. Output saved as D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\TDSM_Job1021395_35078_98_94.tif.
Raster addition completed. Output saved as D:\laz_format_files_Durham_

In [ ]:
## convert the heights of digital models into meters (currently are in feet)

# DEMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEMs'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)

# DSMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DSMs'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DSM'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)

#CDSMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/CDSMs'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/CDSM'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)


DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM\DEM_Job1021395_35078_95_88.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM\DEM_Job1021395_35078_95_91.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM\DEM_Job1021395_35078_95_94.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM\DEM_Job1021395_35078_98_88.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM\DEM_Job1021395_35078_98_91.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM\DEM_Job1021395_35078_98_94.tif
DSM succes

In [ ]:
## function to split tifs into smaller tiles, (1000 x 1000) with a 150 m buffer

def split_dem_into_tiles_with_buffer(input_dem_path, output_folder, file_prefix, tile_size=1000, buffer_size=150, nodata_value=-9999, data_type=gdal.GDT_Float32):
    """
    Splits a DEM into smaller tiles of 1000x1000 meters with a 150-meter buffer.

    Parameters:
        input_dem_path (str): Path to the input DEM file.
        output_folder (str): Folder to save the output tiles.
        tile_size (int): Size of each tile in meters (without buffer).
        buffer_size (int): Buffer size to add to each tile in meters.

    Returns:
        None
    """
    
    # Open the input DEM
    dem_ds = gdal.Open(input_dem_path)
    if dem_ds is None:
        print("Failed to open the input DEM.")
        return

    # Get georeferencing information
    geotransform = dem_ds.GetGeoTransform()
    min_x = geotransform[0]
    max_y = geotransform[3]
    res = geotransform[1]
    pixel_size_x = geotransform[1]
    pixel_size_y = abs(geotransform[5])
    cols = dem_ds.RasterXSize
    rows = dem_ds.RasterYSize

    # Calculate the spatial extent of the DEM
    max_x = min_x + cols * pixel_size_x
    min_y = max_y - rows * pixel_size_y

    # Calculate the size of tiles with buffer
    full_tile_size = tile_size + 2 * buffer_size

    # Create output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through the grid, creating tiles
    x_tiles = math.ceil((max_x - min_x) / tile_size)
    y_tiles = math.ceil((max_y - min_y) / tile_size)

    for i in range(x_tiles):
        for j in range(y_tiles):
            # Calculate the boundaries of the tile with buffer
            tile_min_x = min_x + i * tile_size - buffer_size
            tile_max_x = min_x + (i + 1) * tile_size + buffer_size
            tile_max_y = max_y - j * tile_size + buffer_size
            tile_min_y = max_y - (j + 1) * tile_size - buffer_size

            # Clip to DEM extent
            tile_min_x = max(tile_min_x, min_x)
            tile_max_x = min(tile_max_x, max_x)
            tile_min_y = max(tile_min_y, min_y)
            tile_max_y = min(tile_max_y, max_y)

            # Calculate pixel coordinates of the tile
            offset_x = int((tile_min_x - min_x) / pixel_size_x)
            offset_y = int((max_y - tile_max_y) / pixel_size_y)
            width = int((tile_max_x - tile_min_x) / pixel_size_x)
            height = int((tile_max_y - tile_min_y) / pixel_size_y)

            # Generate output file path
            tile_filename = f"{file_prefix}_tile_{i}_{j}.tif"
            output_tile_path = os.path.join(output_folder, tile_filename)

        # Set the options for gdal.Warp
           # warp_options = gdal.WarpOptions(
           #     format='GTiff',
           #     srcNodata=nodata_value,
           #     dstNodata=nodata_value,
            #    xRes=None,  # Keeps the original resolution
           #     yRes=None,
           #     creationOptions=['COMPRESS=LZW'],  # Compression option (optional)
           #     outputType=data_type
           #         )
            #gdal.Warp(output_tile_path, dem_ds, 
           #       outputBounds = (tile_min_x, tile_min_y, tile_max_x, tile_max_y),format="GTiff", options=warp_options)
            
            #gdal.Translate(output_tile_path, dem_ds, projWin = (tile_min_x, tile_max_y, tile_max_x, tile_min_y), xRes = res, yRes = -res)
            # Extract the tile using gdal.Translate
            gdal.Translate(
                output_tile_path,
                dem_ds,
                srcWin=[offset_x, offset_y, width, height],
               # projWin=[tile_min_x, tile_max_y, tile_max_x, tile_min_y],
                #format='GTiff',
                #xRes=None,  # Keeps the original resolution
                #yRes=None,
                #srcNodata=nodata_value,
                #dstNodata=nodata_value,
                outputType=gdal.GDT_Float32,
                creationOptions=["COMPRESS=LZW"]
           )

            print(f"Created tile: {output_tile_path}")

    # Close the DEM dataset
    dem_ds = None
    print("DEM splitting completed.")

In [47]:
f"{file_prefix}_tile__.tif"

'DEM_Job1021395_35078_95_88_tile__.tif'

In [42]:
input_dem_path[-30:-4]

'DEM_Job1021395_35078_95_88'

In [ ]:
##########################################################################################
##### Splitting tiles in meters                                                      #####
##### the qgis umep one needs meters                                                 #####
##########################################################################################

## DSMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DSM'
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
for input_dem_path in tif_files:
    file_prefix = input_dem_path[-30:-4]
    output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DSM\\tiles_1kby1k_150buffer'
    split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, file_prefix = file_prefix)


## CDSMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\CDSM'
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
for input_dem_path in tif_files:
    file_prefix = input_dem_path[-31:-4]
    output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\CDSM\\tiles_1kby1k_150buffer'
    split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, file_prefix = file_prefix)

## DEMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DEM'
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
for input_dem_path in tif_files:
    file_prefix = input_dem_path[-30:-4]
    output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DEM\\tiles_1kby1k_150buffer'
    split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, file_prefix = file_prefix)


Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_0.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_1.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_2.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_3.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_4.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_5.tif
Created tile: D:\laz_format_files_

In [23]:
##########################################################################################
##### For the files in feet, as this is what the gpu svf calculator is set up to use #####
##### the qgis umep one needs meters                                                 #####
##########################################################################################

## DSMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs'
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
for input_dem_path in tif_files:
    file_prefix = input_dem_path[-30:-4]
    output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs\\tiles_1kby1k_150buffer'
    split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, file_prefix = file_prefix)


## TDSMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\TDSMs'
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
for input_dem_path in tif_files:
    file_prefix = input_dem_path[-31:-4]
    output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\TDSMs\\tiles_1kby1k_150buffer'
    split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, file_prefix = file_prefix)

C:\Users\UEA_Laptop\.conda\envs\ddl_solweig\Lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_0.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_1.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_2.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_3.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_4.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_1kby1k_150buffer\DSM_Job1021395_35078_95_88_tile_0_5.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles

In [25]:
## CDSMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\CDSMs'
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
for input_dem_path in tif_files:
    file_prefix = input_dem_path[-31:-4]
    output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\CDSMs\\tiles_1kby1k_150buffer'
    split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, file_prefix = file_prefix)

Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs\tiles_1kby1k_150buffer\CDSM_Job1021395_35078_95_88_tile_0_0.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs\tiles_1kby1k_150buffer\CDSM_Job1021395_35078_95_88_tile_0_1.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs\tiles_1kby1k_150buffer\CDSM_Job1021395_35078_95_88_tile_0_2.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs\tiles_1kby1k_150buffer\CDSM_Job1021395_35078_95_88_tile_0_3.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs\tiles_1kby1k_150buffer\CDSM_Job1021395_35078_95_88_tile_0_4.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs\tiles_1kby1k_150buffer\CDSM_Job1021395_35078_95_88_tile_0_5.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\C

In [26]:
## DEMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DEMs'
tif_files = glob.glob(os.path.join(input_folder, "*.tif"))
for input_dem_path in tif_files:
    file_prefix = input_dem_path[-30:-4]
    output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DEMs\\tiles_1kby1k_150buffer'
    split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, file_prefix = file_prefix)

Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_0.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_1.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_2.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_3.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_4.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_5.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles

In [27]:
## convert the heights of digital models into meters (currently are in feet)

# DEMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DEMs/tiles_1kby1k_150buffer'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM/tiles_1kby1k_150buffer'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)

# DSMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DSMs/tiles_1kby1k_150buffer'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DSM/tiles_1kby1k_150buffer'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)

#CDSMs
feet_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/CDSMs/tiles_1kby1k_150buffer'
meter_file_path =  'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/CDSM/tiles_1kby1k_150buffer'
#get a list of the filenames
file_name_list = []
for file_name in os.listdir(feet_file_path):
    if file_name.endswith(".tif"):
        file_name_list.append(file_name)

for dm_filename in file_name_list:
    input_dm = os.path.join(feet_file_path, dm_filename)
    output_dm = os.path.join(meter_file_path, dm_filename)
    convert_dsm_feet_to_meters(input_dm, output_dm)


DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM/tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_0.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM/tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_1.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM/tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_10.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM/tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_2.tif
DSM successfully converted to meters and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/Meters/DEM/tiles_1kby1k_150buffer\DEM_Job1021395_35078_95_88_tile_0_3.tif
DSM successful

In [46]:
# Merge the DMs into one and then split into individual files
## creating 1 km tiles
## DEMS
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DEMs\\Merged_DEM\\merged_DEM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DEMs\\tiles_1kby1k_150buffer'
split_dem_into_tiles_with_buffer(merged_output_path, output_folder, tile_size=1000, buffer_size=150, nodata_value=-9999, data_type=gdal.GDT_Float32)

## DSMS
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs\\Merged_DSM\\merged_DSM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs\\tiles_1kby1k_150buffer'
split_dem_into_tiles_with_buffer(merged_output_path, output_folder, tile_size=1000, buffer_size=150, nodata_value=-9999, data_type=gdal.GDT_Float32)


## CDSMS
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\CDSMs\\Merged_CDSM\\merged_CDSM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\CDSMs\\tiles_1kby1k_150buffer'
split_dem_into_tiles_with_buffer(merged_output_path, output_folder, tile_size=1000, buffer_size=150, nodata_value=-9999, data_type=gdal.GDT_Float32)


Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\tile_0_0.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\tile_0_1.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\tile_0_2.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\tile_0_3.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\tile_0_4.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\tile_0_5.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k_150buffer\tile_0_6.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEMs\tiles_1kby1k

In [4]:
## Merge the DEM and CDSM, as the ray tracing algorithm wants it in this format to do the skyview factors 
## (if using the DEM in qgis through UMEP, you need to have the CDSM with no DEM)
CDSM_folder =  'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\CDSMs\\tiles_1kby1k_150buffer'
# get the tile filenames
CDSM_files =  []
for file_name in os.listdir(CDSM_folder):
    if file_name.endswith(".tif"):
        CDSM_files.append(file_name)
DEM_folder =  'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DEMs\\tiles_1kby1k_150buffer'
TDSM_folder =  'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\TDSMs\\tiles_1kby1k_150buffer'
#DEM_files = glob.glob(os.path.join(DEM_folder, "*.tif"))

for filename in CDSM_files:
    CDSM_filepath = os.path.join(CDSM_folder,filename)
    DEM_filepath = os.path.join(DEM_folder,filename)

    ### merge the DEM and trees
    input_files = [ DEM_filepath, CDSM_filepath]
    TDSM_filepath = os.path.join(TDSM_folder, filename)
    merge_rasters(input_files, TDSM_filepath, nodata_value=-9999, data_type=gdal.GDT_Float32)

Raster files merged and saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\tiles_1kby1k_150buffer\tile_0_0.tif
Raster files merged and saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\tiles_1kby1k_150buffer\tile_0_1.tif
Raster files merged and saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\tiles_1kby1k_150buffer\tile_0_10.tif
Raster files merged and saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\tiles_1kby1k_150buffer\tile_0_11.tif
Raster files merged and saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\tiles_1kby1k_150buffer\tile_0_12.tif
Raster files merged and saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TDSMs\tiles_1kby1k_150buffer\tile_0_13.tif
Raster files merged and saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\TD

In [5]:
## Want to remove the files which don't work (for now, until I work out how to fix them)
## check if they are in the dsm folder and if not move them to the 'don't work' folder

DSM_folder =  'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs\\tiles_1kby1k_150buffer'
TDSM_folder =  'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\TDSMs\\tiles_1kby1k_150buffer'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\TDSMs\\tiles_1kby1k_150buffer\\tiles_with_nans_or_dont_work'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through files in the tdsm folder
for file_name in os.listdir(TDSM_folder):
    tdsm_file_path = os.path.join(TDSM_folder, file_name)

    # Check if it's a file (skip directories)
    if os.path.isfile(tdsm_file_path):
        # Check if the file exists in the dsm folder
        dsm_file_path = os.path.join(DSM_folder, file_name)
        if not os.path.exists(dsm_file_path):
            # If the file is not in the dsm folder, move it to the output folder
            shutil.move(tdsm_file_path, os.path.join(output_folder, file_name))
            print(f"Moved: {file_name}")
        else:
            print(f"File exists in dsm: {file_name}")

Moved: tile_0_0.tif
Moved: tile_0_1.tif
Moved: tile_0_10.tif
Moved: tile_0_11.tif
Moved: tile_0_12.tif
Moved: tile_0_13.tif
Moved: tile_0_14.tif
Moved: tile_0_15.tif
Moved: tile_0_16.tif
Moved: tile_0_17.tif
Moved: tile_0_18.tif
Moved: tile_0_19.tif
Moved: tile_0_2.tif
Moved: tile_0_20.tif
Moved: tile_0_21.tif
Moved: tile_0_22.tif
Moved: tile_0_23.tif
Moved: tile_0_24.tif
Moved: tile_0_25.tif
Moved: tile_0_26.tif
Moved: tile_0_27.tif
Moved: tile_0_28.tif
Moved: tile_0_29.tif
Moved: tile_0_3.tif
Moved: tile_0_30.tif
Moved: tile_0_31.tif
Moved: tile_0_32.tif
Moved: tile_0_4.tif
Moved: tile_0_5.tif
Moved: tile_0_6.tif
Moved: tile_0_7.tif
Moved: tile_0_8.tif
Moved: tile_0_9.tif
Moved: tile_10_0.tif
Moved: tile_10_1.tif
Moved: tile_10_10.tif
Moved: tile_10_11.tif
File exists in dsm: tile_10_12.tif
File exists in dsm: tile_10_13.tif
File exists in dsm: tile_10_14.tif
File exists in dsm: tile_10_15.tif
File exists in dsm: tile_10_16.tif
File exists in dsm: tile_10_17.tif
File exists in dsm: t

In [15]:
import rasterio as rio

In [16]:
## checking the files for missing data
#demroot = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DEMs\\tiles_1kby1k_150buffer'
demroot =  'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\CDSMs'

for file in glob.glob(os.path.join(demroot, "*.tif")):
    #demfile = os.path.join(demroot, file)
    demfile = file
    lu_dataset = rio.open(demfile)
    
    lu_bounds = lu_dataset.bounds
    lu_img = lu_dataset.read(1)
    if (lu_img == -9999).any():
        no_data = 'Missing data'
    else: 
        no_data = 'No missing data'
    print('The lu_bounds is:', lu_bounds, file, no_data)

The lu_bounds is: BoundingBox(left=2032546.01, bottom=800715.55, right=2041438.01, top=811647.55) D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs\CDSM_Job1021395_35078_95_88.tif Missing data
The lu_bounds is: BoundingBox(left=2023669.92, bottom=800706.97, right=2032558.92, top=811635.97) D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\CDSMs\CDSM_Job1021395_35078_95_91.tif No missing data


KeyboardInterrupt: 

In [10]:
## fill in the missing data 
input_dsm_path = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\DSM_Job1021395_35078_98_91.tif'

check_nodata(input_dsm_path, nodata_value=-9999)
iteration = 1
new_output_dsm_path = os.path.join(classifications_folder, 'DEM_python_iteration{}.tif'.format(iteration))
fill_nodata(input_dsm_path, new_output_dsm_path, max_distance = 100, smoothing_iterations = 2)

<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_14692\3437437390.py:2: SyntaxWarning: invalid escape sequence '\l'
  input_dsm_path = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\DSM_Job1021395_35078_98_91.tif'


Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEM_python_iteration1.tif


In [11]:
input_dsm_path = 'D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEM_python_iteration1.tif'
check_nodata(input_dsm_path, nodata_value=-9999)

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_14692\3358267644.py:1: SyntaxWarning: invalid escape sequence '\C'
  input_dsm_path = 'D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEM_python_iteration1.tif'


np.False_

In [12]:
## fill in the missing data 
input_dsm_path = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\DSM_Job1021395_36078_01_94.tif'

check_nodata(input_dsm_path, nodata_value=-9999)
iteration = 1
new_output_dsm_path = os.path.join(classifications_folder, 'DEM_python_iteration{}.tif'.format(iteration))
fill_nodata(input_dsm_path, new_output_dsm_path, max_distance = 100, smoothing_iterations = 2)

<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_14692\575297530.py:2: SyntaxWarning: invalid escape sequence '\l'
  input_dsm_path = 'D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\DSM_Job1021395_36078_01_94.tif'


Filled DSM saved to D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEM_python_iteration1.tif


In [13]:
input_dsm_path = 'D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEM_python_iteration1.tif'
check_nodata(input_dsm_path, nodata_value=-9999)

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\UEA_Laptop\AppData\Local\Temp\ipykernel_14692\3358267644.py:1: SyntaxWarning: invalid escape sequence '\C'
  input_dsm_path = 'D:/laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DEM_python_iteration1.tif'


np.False_

array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)

In [36]:
if demfile

'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DEMs\\Merged_DEM\\merged_DEM.tif'

In [ ]:
# Merge the DMs into one and then split into individual files
## DEMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DEM'
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DEM\\merged_DEM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DEM\\tiles_2kby2k_150buffer'
merge_and_split_dems(input_folder, merged_output_path, output_folder)

## NOTE!! For some reason all the files with j = 0 to 4 come out strange, i.e. _i_0, _i_1,_i_2,_i_3,_i_4


Merging input TIFF files...


In [117]:
## DSMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DSM'
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DSM\\merged_DSM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DSM\\tiles_2kby2k_150buffer'
merge_and_split_dems(input_folder, merged_output_path, output_folder)


Merging input TIFF files...
Merged DEM saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\merged_DSM.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_2kby2k_150buffer\tile_0_0.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_2kby2k_150buffer\tile_0_1.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_2kby2k_150buffer\tile_0_2.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_2kby2k_150buffer\tile_0_3.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_2kby2k_150buffer\tile_0_4.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\DSM\tiles_2kby2k_150buffer\tile_0_5.tif
Created tile: D:\laz_format_files_Durham_whole_ar

In [118]:
## CDSMS
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\CDSM'
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\CDSM\\merged_CDSM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\CDSM\\tiles_2kby2k_150buffer'
merge_and_split_dems(input_folder, merged_output_path, output_folder)

Merging input TIFF files...
Merged DEM saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\CDSM\merged_CDSM.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\CDSM\tiles_2kby2k_150buffer\tile_0_0.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\CDSM\tiles_2kby2k_150buffer\tile_0_1.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\CDSM\tiles_2kby2k_150buffer\tile_0_2.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\CDSM\tiles_2kby2k_150buffer\tile_0_3.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\CDSM\tiles_2kby2k_150buffer\tile_0_4.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\Meters\CDSM\tiles_2kby2k_150buffer\tile_0_5.tif
Created tile: D:\laz_format_files_Durham_

In [ ]:
###################################
## 1km by 1km tiles, 150m buffer ##
###################################

## DEMS
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DEM\\merged_DEM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DEM\\tiles_1kby1k_150buffer'
input_dem_path = merged_output_path
split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, buffer_size=150, nodata_value=-9999, data_type=gdal.GDT_Float32)

## DSMS
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DSM\\merged_DSM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\DSM\\tiles_1kby1k_150buffer'
input_dem_path = merged_output_path
split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, buffer_size=150, nodata_value=-9999, data_type=gdal.GDT_Float32)

## CDSMS
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\CDSM\\merged_CDSM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\Meters\\CDSM\\tiles_1kby1k_150buffer'
input_dem_path = merged_output_path
split_dem_into_tiles_with_buffer(input_dem_path, output_folder, tile_size=1000, buffer_size=150, nodata_value=-9999, data_type=gdal.GDT_Float32)



In [119]:
## DSMS in feet
input_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs'
merged_output_path = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs\\merged_DSM.tif'
output_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs\\tiles_2kby2k_150buffer'
merge_and_split_dems(input_folder, merged_output_path, output_folder)


Merging input TIFF files...
Merged DEM saved to D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\merged_DSM.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_2kby2k_150buffer\tile_0_0.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_2kby2k_150buffer\tile_0_1.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_2kby2k_150buffer\tile_0_2.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_2kby2k_150buffer\tile_0_3.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_2kby2k_150buffer\tile_0_4.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs\tiles_2kby2k_150buffer\tile_0_5.tif
Created tile: D:\laz_format_files_Durham_whole_area_2015\Central_Tiles\Classifications\DSMs

In [11]:
## get the bounding boxes for the tiles, so the air temp/ met data can be linked

# Define the source folder containing the .tif files
source_folder = 'D:\\laz_format_files_Durham_whole_area_2015\\Central_Tiles\\Classifications\\DSMs\\tiles_1kby1k_150buffer'

# Initialize a list to store file information
data = []

# Transformer to convert from NAD83 (2011) to WGS84 (latitude and longitude)
transformer = Transformer.from_crs("EPSG:26917", "EPSG:4326", always_xy=True)  # NAD83(2011) -> WGS84
# Loop through all .tif files in the folder
for filename in os.listdir(source_folder):
    if filename.endswith(".tif"):
        file_path = os.path.join(source_folder, filename)
        
        # Open the raster file
        with rasterio.open(file_path) as src:
            # Get the bounding box in NAD83 (2011)
            bounds = src.bounds  # left, bottom, right, top
            
            # Transform the bounding box to WGS84
            left, bottom = transformer.transform(bounds.left, bounds.bottom)
            right, top = transformer.transform(bounds.right, bounds.top)
            
            # Append the data
            data.append({
                "filename": filename,
                "left": left,
                "bottom": bottom,
                "right": right,
                "top": top
            })

# Create a DataFrame from the collected data
tiles_bounding_boxes_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file (optional)
#output_file = "bounding_boxes.csv"
#df.to_csv(output_file, index=False)

# Display the DataFrame
print(tiles_bounding_boxes_df)



           filename       left    bottom      right       top
0    tile_10_12.tif -67.318503  7.211715 -67.306714  7.222789
1    tile_10_13.tif -67.318772  7.202928 -67.306984  7.214002
2    tile_10_14.tif -67.319040  7.194140 -67.307253  7.205215
3    tile_10_15.tif -67.319309  7.185353 -67.307522  7.196428
4    tile_10_16.tif -67.319576  7.176565 -67.307790  7.187641
..              ...        ...       ...        ...       ...
550   tile_9_27.tif -67.331297  7.080165 -67.319518  7.091246
551   tile_9_28.tif -67.331561  7.071377 -67.319782  7.082458
552   tile_9_29.tif -67.331824  7.062589 -67.320046  7.073671
553   tile_9_30.tif -67.332087  7.053801 -67.320310  7.064883
554   tile_9_31.tif -67.332350  7.045013 -67.320573  7.056096

[555 rows x 5 columns]


In [12]:
tiles_bounding_boxes_df

,filename,left,bottom,right,top
0,tile_10_12.tif,-67.318503,7.211715,-67.306714,7.222789
1,tile_10_13.tif,-67.318772,7.202928,-67.306984,7.214002
2,tile_10_14.tif,-67.319040,7.194140,-67.307253,7.205215
3,tile_10_15.tif,-67.319309,7.185353,-67.307522,7.196428
4,tile_10_16.tif,-67.319576,7.176565,-67.307790,7.187641
...,...,...,...,...,...
550,tile_9_27.tif,-67.331297,7.080165,-67.319518,7.091246
551,tile_9_28.tif,-67.331561,7.071377,-67.319782,7.082458
552,tile_9_29.tif,-67.331824,7.062589,-67.320046,7.073671
553,tile_9_30.tif,-67.332087,7.053801,-67.320310,7.064883


In [9]:
left, bottom 

(2023632.65, 801338.78)

In [ ]:
input_files

In [157]:
def get_raster_size(raster_path):
    with rasterio.open(raster_path) as src:
        width = src.width
        height = src.height
        return width, height

raster_file = "D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DSMs/DSM_Job1021395_35078_98_94.tif"
width, height = get_raster_size(raster_file)

print("Raster width:", width)
print("Raster height:", height)

Raster width: 8882
Raster height: 10927


In [158]:
with rasterio.open(raster_file) as src:
    # Get basic metadata
    print("Width:", src.width)
    print("Height:", src.height)
    print("Number of bands:", src.count)
    print("Coordinate Reference System (CRS):", src.crs)
    print("Bounding box:", src.bounds)
    print("Data type:", src.dtypes)
    print("Driver:", src.driver)

Width: 8882
Height: 10927
Number of bands: 1
Coordinate Reference System (CRS): COMPD_CS["NAD83(2011) / North Carolina (ftUS); NAVD88 height (ftUS)",PROJCS["NAD83(2011) / North Carolina (ftUS)",GEOGCS["NAD83(2011)",DATUM["NAD83_National_Spatial_Reference_System_2011",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1116"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","6318"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",33.75],PARAMETER["central_meridian",-79],PARAMETER["standard_parallel_1",36.1666666666667],PARAMETER["standard_parallel_2",34.3333333333333],PARAMETER["false_easting",2000000],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","6543"]],VERT_CS["NAVD88 height (ftUS)",VERT_DATUM["North American Vertical Datum 1988",2005,AUTHORI

In [89]:
### merge the DSM and trees

input_files = [
    'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/DSMs\\DSM_Job1021395_35078_98_94.tif',
    'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/trees_Job1021395_35078_98_94.tif'
]

folder_for_trees_DEM = 'D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/trees_plus_DEM'
new_filename = 'trees_DSM_{}.tif'.format(fileinfo)

output_file = os.path.join(folder_for_trees_DEM, new_filename)

merge_rasters(input_files, output_file, nodata_value=-9999, data_type=gdal.GDT_Float32)


Raster files merged and saved to D:/laz_format_files_Durham_whole_area_2015/Central_Tiles/Classifications/trees_plus_DEM\trees_DSM_Job1021395_35078_98_94.tif
